In [1]:
#|default_exp dicom

# DICOMSDL

Before we used [pydicom](https://pydicom.github.io) to load and work with DICOM data. However, the [dicomsdl](https://github.com/tsangel/dicomsdl) library is faster, since it is written in C++. It also exposes and API in Python. Let's time it:

In [1]:
!pip install -Uqq dicomsdl

In [2]:
import pydicom
import dicomsdl

In [3]:
import sys
sys.path.append("../../lib")

In [4]:
import os

In [5]:
series_path_dicom = f"{os.environ['RSNA_IAD_DATA_DIR']}/series"
series_uid_l = os.listdir(series_path_dicom)
len(series_uid_l)

4348

In [6]:
def dicom_serie_load(series_base_path, serie_uid, load_callback):

    serie_path = f"{series_base_path}/{serie_uid}"
    instances_filename = os.listdir(serie_path)
    n_instances = len(instances_filename)

    ds_l = [None] * n_instances
    for i, instance_filename in enumerate(instances_filename):
        ds_l[i] = load_callback(f"{serie_path}/{instance_filename}")

    return ds_l

In [7]:
import random

series_uid_sample_l = random.sample(series_uid_l, 10)

In [8]:
%%timeit
for serie_uid in series_uid_sample_l:
    ds_l = dicom_serie_load(series_path_dicom, serie_uid, pydicom.dcmread)

1.1 s ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
for serie_uid in series_uid_sample_l:
    ds_l = dicom_serie_load(series_path_dicom, serie_uid, dicomsdl.open)

360 ms ± 7.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The `dicomsdl` library is ~3x faster at loading the DICOM files data, as it is stored. Now let's also test obtaining the pixel array data:

In [10]:
def dicom_serie_load_volume(series_base_path, serie_uid, load_callback):

    serie_path = f"{series_base_path}/{serie_uid}"
    instances_filename = os.listdir(serie_path)
    n_instances = len(instances_filename)

    slices_l = [None] * n_instances
    for i, instance_filename in enumerate(instances_filename):
        slices_l[i] = load_callback(f"{serie_path}/{instance_filename}")

    return slices_l

In [11]:
def pydicom_callback(x):
    return pydicom.dcmread(x).pixel_array

def dicomsdl_callback(x):
    return dicomsdl.open(x).pixelData(storedvalue=True)

In [12]:
%%timeit
for serie_uid in series_uid_sample_l:
    slices_l = dicom_serie_load_volume(series_path_dicom, serie_uid, pydicom_callback)

18.3 s ± 45.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
for serie_uid in series_uid_sample_l:
    slices_l = dicom_serie_load_volume(series_path_dicom, serie_uid, dicomsdl_callback)

1.89 s ± 5.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The `dicom_sdl` is ~10x faster when also obtaining the pixel array data, in sequential loading. Now let's test this when loading series in parallel:

In [14]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

def dicom_series_load_volume(base_path_dicoms, series_uid, max_workers, load_callback):

    n_instances = len(series_uid)
    volumes_l = [None] * n_instances
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(dicom_serie_load_volume, base_path_dicoms, serie_uid, load_callback) for serie_uid in series_uid]
        for i, future in enumerate(as_completed(futures)):
            volumes_l[i] = future.result()

    return volumes_l

In [17]:
%%timeit
slices_l = dicom_series_load_volume(series_path_dicom, series_uid_sample_l, 4, pydicom_callback)

7.65 s ± 57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
slices_l = dicom_series_load_volume(series_path_dicom, series_uid_sample_l, 4, dicomsdl_callback)

2.58 s ± 37.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


With parallelization, `dicomsdl` is ~3x faster.